## Bisection Method

**Step 1.** Find segment $[a,b]$ on which all eigenvalues are located. For example, $[-\|\boldsymbol{A}\|_1,+\|\boldsymbol{A}\|_1]$

**Step 2.** For each $i=0,\dots,N$ we:
- compute $c = \frac{a+b}{2}$;
- find sequence $\{\sigma_k\}_{k=0}^n$ according to:
$$
\sigma_0 = 1, \; \sigma_1 = \alpha_1 - c, \; \sigma_{k+1} = (\alpha_{k+1}-c)\sigma_k - \beta_k^2 \sigma_{k-1};
$$
- find number of sign changes $s$ in sequence $\{\sigma_k\}_{k=0}^n$;
- If $s < r$, set $a=c$, otherwise $b=c$.

#### Task
Make up a symmetric tridiagonal matrix $\boldsymbol{A} \in \mathbb{R}^{4 \times 4}$ and apply _bisection method_ to find its $3^{\text{rd}}$ eigenvalue. 

In [26]:
# Import numpy and set print options
import numpy as np

np.set_printoptions(precision=4)

In [40]:
def tridiagonal_symmetric_norm(alpha, beta):
    """
    tridiagonal_norm find L_infty norm of tridiagonal symmetric matrix
    Input:
    alpha - diagonal entries
    beta - entries above and below the diagona
    
    Output:
    L_infty norm of matrix
    """
    
    n = len(alpha) # Size of a matrix 
    # Finding max of rows sum containing three elements
    rows_max = max([abs(beta[j-1]) + abs(alpha[j]) + abs(beta[j]) for j in range(1, n-1)])
    row_1 = abs(alpha[0]) + abs(beta[0]) # Finding the first row's sum
    row_n = abs(alpha[-1]) + abs(beta[-1]) # Finding the last row's sum
    return max([row_1, rows_max, row_n])

def offset_minors(alpha, beta, offset):
    """
    minors finds minors of tridiagonal symmetric matrix with a given offset
    Input:
    alpha - diagonal entries
    beta - entries above and below the diagona
    offset - offset (value of c in algorithm above)
    
    Output:
    A slice of length n+1 with given minors
    """
    
    n = len(alpha) # Finding a size of matrix A
    sigma = np.empty(n+1) # Filling sigmas
    sigma[0] = 1 # First sigma is 1
    sigma[1] = alpha[0] - offset # Setting initial sigma[1] 
    for i in range (2,n+1) :
        sigma[i] = (alpha[i-1]-offset)*sigma[i-1] - beta[i-2]**2 * sigma[i-2]
    return sigma

def sign_changes(sigma):
    """
    sign_changes finds number of sign changes in a given array of sigmas
    Input:
    sigma - array of sigmas
    Output:
    number of sign changes
    """
    
    n = len(sigma)-1 # Size of matrix A is len(sigma) - 1
    sigma = np.where(sigma<0, -1, 1) # Store signs only in array of sigma
    # Filtering when product of two adjacent elements is negative and sum this number
    sign_changes = (sigma[1:]*sigma[:n] < 0).sum()
    return sign_changes

def bisection_method(alpha, beta, r=0, steps_number=30):
    """
    bisection_method finds r^th eigenvalue of A using bisection method
    
    Input:
    alpha - diagonal entries
    beta - entries above and below the diagona
    r - eigenvalue number to find
    steps_number - (optional) number of steps to make
    
    Output:
    Searched eigenvalue
    """
    
    n = len(alpha) # Finding a size of alpha
    b = tridiagonal_symmetric_norm(alpha, beta) # Finding norm of matrix
    a = -b # Making segment [-norm, +norm]
    c = 0 # Number c in algorithm; will approach a searched eigenvalue
    
    for i in range(steps_number):
        c = (a+b)/2 # Finding a middle point of segment [a,b]
        minors = offset_minors(alpha, beta, offset=c) # Finding matrix minors
        sign_changes_num = sign_changes(minors) # Finding number of sign changes
        if sign_changes_num < r:
            a = c
        else:
            b = c
    return c

Let us check our results on a matrix
$$
\boldsymbol{A} = \begin{bmatrix}
2 & 1 & 0 & 0 \\
1 & 2 & 1 & 0 \\
0 & 1 & 2 & 1 \\
0 & 0 & 1 & 2
\end{bmatrix}
$$

In [41]:
# Find eigenvalues using np.linalg.eigvals function
expected_lambdas = np.linalg.eigvals(np.array([
    [2, 1, 0, 0],
    [1, 2, 1, 0],
    [0, 1, 2, 1],
    [0, 0, 1, 2]
], dtype=np.float64))
# Print expected result
print('Eigenvalues of a given matrix are {}'.format(expected_lambdas))

# Forming an array of alphas and betas
alpha = np.full(4, 2.0)
beta = np.full(3, 1.0)

for i in range(1, 5):
    # Finding eigenvalues according to our algorithm and printing a result
    actual_lambda = bisection_method(alpha, beta, r=i, steps_number=30)
    print('The eigenvalue #{} of a given matrix is {}'.format(i, actual_lambda))

Eigenvalues of a given matrix are [3.618 2.618 0.382 1.382]
The eigenvalue #1 of a given matrix is 0.3819660171866417
The eigenvalue #2 of a given matrix is 1.3819660171866417
The eigenvalue #3 of a given matrix is 2.6180339828133583
The eigenvalue #4 of a given matrix is 3.6180339828133583


Now, let us apply our method on a matrix
$$
\boldsymbol{A} = \begin{bmatrix}
1 & 3 & 0 & 0 \\
3 & 2 & 5 & 0 \\
0 & 5 & 3 & 7 \\
0 & 0 & 7 & 4
\end{bmatrix}
$$

In [42]:
# Defining diagonal elements
alpha = np.array([1.0, 2.0, 3.0, 4.0], dtype=np.float64)
beta = np.array([3.0, 5.0, 7.0], dtype=np.float64)

# Using np.linalg.eigvals function to find the third eigenvalue:
sorted_expected_eigvals = sorted(np.linalg.eigvals(np.array([
    [1, 3, 0, 0],
    [3, 2, 5, 0],
    [0, 5, 3, 7],
    [0, 0, 7, 4]
], dtype=np.float64)))
print('Expected third eigenvalue is {}'.format(sorted_expected_eigvals[2]))

# Applying our algorithm to find third eigenvalue
actual_eigval = bisection_method(alpha, beta, r=3, steps_number=30)
print('Actual third eigenvalue is {}'.format(actual_eigval))

Expected third eigenvalue is 4.417669790511108
Actual third eigenvalue is 4.417669801041484
